In [98]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [99]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [100]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [101]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time
import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report
import joblib

On va d'abord extraire les titres des articles d'un site d'information français, nous avons choisi le site [Le Monde](https://www.lemonde.fr/)

In [102]:
url = 'https://www.lemonde.fr/'

In [103]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}
response = requests.get(url, headers=headers)

In [104]:
if response.status_code == 200:
    # Utiliser Beautiful Soup pour extraire le contenu HTML de la page
    soup = BeautifulSoup(response.content, 'html.parser')

    # Exemple : Extraire tous les titres d'articles
    titles = soup.find_all('p', class_='article__title')

    # Afficher les titres
    for title in titles:
        print(title.text.strip())
        print("\n")
else:
    print(f"La requête a échoué avec le code d'état {response.status_code}")

La Peugeot e-308, voiture électrifiée plutôt qu’électrique


Internet : « En matière de messagerie, le gouvernement monte d’un cran ses exigences »


Défense : « des carnets de commandes pleins, mais des ventes d’armes qui n’ont pas suivi en 2022 »


« Il n’existe aucune preuve de l’impact d’une formation de sensibilisation aux enjeux climatiques sur les comportements »


L’hôpital Al-Shifa, centre de commandement du Hamas ? Ce que montrent les images


« Dubaï connexion, comment blanchir cinquante millions de dollars » : découvrez le troisième épisode de notre série vidéo « Narco Business »


Les images de l’hôpital local Nasser, au sud de la bande de Gaza, débordé par le nombre de blessés


Bande de Gaza : avant la guerre, un territoire à l’agonie


Dette : la France plus que jamais sous surveillance


La « désoccidentalisation », ou l’affirmation continue des pays du Sud


Laïcité : « La tolérance des jeunes Français est bien supérieure à celle de leurs aînés »


Asmaa Al-Ghoul, jou

On va maintenant extraire les titres des articles d'un site d'information international, nous avons choisi le site américain [The Washington Post](https://www.washingtonpost.com/)

In [105]:
url2 = 'https://www.washingtonpost.com/'

In [106]:
response2 = requests.get(url2, headers=headers)

In [107]:
if response2.status_code == 200:
    # Utiliser Beautiful Soup pour extraire le contenu HTML de la page
    soup = BeautifulSoup(response2.content, 'html.parser')

    # Exemple : Extraire tous les titres d'articles
    titles = soup.find_all('h2', class_='wpds-c-iiQaMf wpds-c-iiQaMf-ihElTZa-css')
    titles = titles + soup.find_all('h2', class_='wpds-c-iiQaMf wpds-c-iiQaMf-ibYgSwf-css')

    # Afficher les titres
    for title in titles:
        print(title.text.strip())
        print("\n")
else:
    print(f"La requête a échoué avec le code d'état {response.status_code}")

Readers react to Post report about devastation from AR-15 shootings


Read the story: Terror on repeat


Your hair care routine might carry health risks, this study says


Work Advice: This ‘holiday’ office party is all Christmas, all the way


What you can do to boost your covid and flu shots’ effectiveness


No one really knows how Facebook works. This book might help.


A Trump dictatorship is increasingly inevitable. We should stop pretending.


Americans are boringly predictable in smartphones. Here’s why.


The U.S. government gives billions to Musk’s companies. It can’t quit him.


Biden’s got a new set of orders for Obamacare. Is it his last?


The strange tale of Scotland’s buried Egyptian treasures


The 10 best video game gifts to give this year


10 great books for gift-giving this year


10 great tech gift ideas


Give the gift of food this year with these 10 great ideas


10 great toys to give this holiday season


10 great travel gifts for the road ahead


2023 Holiday G

On va maintenant extraire les commentaires d'une vidéo [Youtube](https://www.youtube.com/) dont le lien est passé en paramètres

In [135]:
def youtube_scraping(link) :

    target_url = [link]

    s = Service(r"C:\Program Files (x86)\chromedriver.exe")

    for url in target_url:
        driver = webdriver.Chrome(service=s)
        driver.get(url)

    driver.maximize_window()
    time.sleep(4)
    driver.execute_script("window.scrollTo(0, 1000);")
    for i in range(0,10):
        time.sleep(2)
        driver.execute_script("window.scrollTo(0, 15000);")

    comments_section = driver.find_element(By.XPATH,'//*[@id="comments"]')

    comments_html = comments_section.get_attribute('innerHTML')

    soup = BeautifulSoup(comments_html, 'html.parser')

    # extraction du texte des commentaires
    comments = [comment.text for comment in soup.find_all('yt-formatted-string', {'class': 'style-scope ytd-comment-renderer'})]

    # affichage des commentaires
    for comment in comments :
        print(comment)
        print("\n")
    driver.close()
    
    return comments

Desormais, on va effectuer un analyse de sentiment sur les réponses qu'on a obtenu ci dessus. 
L'objectif est de déterminer un ratio réponses positives/réponses négatives (en pourcentage) dans le but de remplacer les likes/dislikes, étant donné que YouTube a récemment supprimé la fonctionnalité de dislike.

In [146]:
df = pd.read_csv("comments.csv")
df = df[df.Sentiment != 1.0]
X = df.Comment
y = df.Sentiment
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

Ici, on a choisi un modèle de type SVC. Dans tous les cas, il nous fallait un modèle capable de catégoriser les data et parmi ceux testés, c'est SVC qui a fourni le meilleur résultat. On utilise un pipeline pour coupler le modèle SVC avec un TfidfVectorizer permettant de traiter les données textuelles.

In [147]:
pipeline = Pipeline([('tfidf', TfidfVectorizer()),
                     ('model', SVC())])

parameters = {"tfidf__ngram_range": [(1, 1)],
              "model__kernel": ["rbf"],
              "model__C": np.logspace(-5,5,3)}

pipeline_gs = GridSearchCV(pipeline, parameters)

pipeline_gs.fit(X_train, y_train)
print(pipeline_gs.best_params_)
y_pred = pipeline_gs.predict(X_test)

{'model__C': 100000.0, 'model__kernel': 'rbf', 'tfidf__ngram_range': (1, 1)}


In [148]:
print(classification_report(y_test, y_pred, digits=3))

print(df.Sentiment.value_counts())

              precision    recall  f1-score   support

         0.0      0.874     0.477     0.618       465
         2.0      0.903     0.986     0.943      2289

    accuracy                          0.900      2754
   macro avg      0.888     0.732     0.780      2754
weighted avg      0.898     0.900     0.888      2754

Sentiment
2.0    11432
0.0     2338
Name: count, dtype: int64


In [158]:
commentaires = youtube_scraping("https://www.youtube.com/watch?v=IscGtF_A14A")

# Utilisez le modèle pour prédire les labels
new_predictions = pipeline_gs.predict(commentaires)

Hey guys, hope you are enjoying the video. The amount of work to put this together was really huge so if you want to support me you can do it here: https://www.paypal.com/donate/?hosted_button_id=AVE5MR2QYKZS8

Also don't forget to like and share! Thank you guys :)


Wow, incredible effort to create this video. Keep it up!


The first commentator called it. "Remember the name, Messi". From there he became one of the best. Wonder if he thought he'd be this renowned in 2005


the fact that the first Messi goal ever was after a Ronaldinho assist is just beautiful


The greatest football player of all time. Plain and simple !!


I feel bad for people who didn't watch him week in week out. 
What he's done is impossible to describe in words.


I love how he points to the players who give him the assist in most of his goals, giving them the credit of it. That demonstrate his enormous humility. 


Sensation work! What a Legendary player Leo Messi  
PS. CR7 Fan 🫡


Best goals Timestamps
0:08 Fi

In [159]:
nb_0 = 0
nb_1 = 0
for phrase, prediction in zip(commentaires, new_predictions):
    if prediction==0.0 :
        nb_0 += 1
    else :
        nb_1 += 1
print("Commentaires positifs :", round(nb_1/(nb_0+nb_1) * 100,2), "%")
print("Commentaires négatifs :", round(nb_0/(nb_0+nb_1) * 100,2), "%")

Commentaires positifs : 99.29 %
Commentaires négatifs : 0.71 %
